In [1]:
import boto3
import json
import csv
import dateutil.parser as parser
from time import sleep
from datetime import datetime

In [2]:
kinesis_stream = "us-accidents-streams-dev"
raw_file = r"C:\Users\as1\OneDrive - Capgemini\Documents\data_science\projectpro\de\Build a real-time Streaming Data Pipeline using Flink and Kinesis\Data\US_Accidents_Dec20_updated.csv"
partition_key = "Severity"

In [3]:
kinesis_client = boto3.client("kinesis", region_name="ap-south-1")

In [4]:
def data_stream_simulator():
    
    #read the csv file
    with open(raw_file, 'r') as accident_data:
        lines = accident_data.read().split('\n')
    
    for row in csv.DictReader(lines[0:10]):
        try:
            line_json = json.dumps(row)
            json_load = json.loads(line_json)

            start_time_raw = parser.parse(json_load["Start_Time"])
            start_time_iso = start_time_raw.isoformat()
            json_load.update({"Start_Time": start_time_iso})

            end_time_raw = parser.parse(json_load["End_Time"])
            end_time_iso = end_time_raw.isoformat()
            json_load.update({"End_Time": end_time_iso})

            weather_time_raw = parser.parse(json_load['Weather_Timestamp'])
            weather_time_iso = weather_time_raw.isoformat()
            json_load.update({'Weather_Timestamp':weather_time_iso})

            json_load['Txn_Timestamp'] = datetime.now().isoformat()

            response = kinesis_client.put_record(StreamName=kinesis_stream, Data=json.dumps(json_load, indent=4), 
                                      PartitionKey=str(json_load[partition_key]))

            sleep(1)
            print(json_load[partition_key])
            print(response)
        except Exception as e:
            print(f"Error: {e}")
        

In [7]:
data_stream_simulator()

3
{'ShardId': 'shardId-000000000003', 'SequenceNumber': '49638566425682099547892571947386153316344595309012189234', 'ResponseMetadata': {'RequestId': 'cc79ddb2-c063-6eca-9004-b847c1924930', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'cc79ddb2-c063-6eca-9004-b847c1924930', 'x-amz-id-2': '8GYhAiushV/h9kqCAWQmW3K4YBuoFJGfqAvAQpCibh4RAKaVEamlOfs3xU5gCV+Mdndpjm3e7LZiGpuipYy9gkr+rajdbkDlB6+eUbGGjI4=', 'date': 'Sat, 04 Mar 2023 15:28:53 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '110'}, 'RetryAttempts': 0}}
2
{'ShardId': 'shardId-000000000003', 'SequenceNumber': '49638566425682099547892571947387362242164210006906372146', 'ResponseMetadata': {'RequestId': 'f7f67fad-88c1-93f4-ab8b-1a5b8930b40e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'f7f67fad-88c1-93f4-ab8b-1a5b8930b40e', 'x-amz-id-2': 'kdEbCpEG0dYA5BLr0n5Ervyi9Hfdwtopzv2joUU8coGNP1ppqSLwZ2YLSjbVvkwgzeNHYw4nVxdNoaYT1LF5ZME0npj4TnkHXx3BD7dia6k=', 'date': 'Sat, 04 Mar 2023 15:28:54 G